<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

--2022-07-21 20:53:02--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2022-07-21 20:53:02--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2022-07-21 20:53:02--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['blink-182.txt',
 'bruce-springsteen.txt',
 'nickelback.txt',
 'nicki-minaj.txt',
 'bieber.txt',
 'bjork.txt',
 'lady-gaga.txt',
 'Kanye_West.txt',
 'bob-marley.txt',
 'janisjoplin.txt',
 'dickinson.txt',
 'nursery_rhymes.txt',
 'johnny-cash.txt',
 'dj-khaled.txt',
 'adele.txt',
 'notorious_big.txt',
 'ludacris.txt',
 'paul-simon.txt',
 'cake.txt',
 'radiohead.txt',
 'prince.txt',
 'amy-winehouse.txt',
 'beatles.txt',
 'r-kelly.txt',
 'Lil_Wayne.txt',
 'lorde.txt',
 'kanye.txt',
 'dr-seuss.txt',
 'drake.txt',
 'lin-manuel-miranda.txt',
 'al-green.txt',
 'nirvana.txt',
 'joni-mitchell.txt',
 'patti-smith.txt',
 'britney-spears.txt',
 'kanye-west.txt',
 'eminem.txt',
 'disney.txt',
 'bob-dylan.txt',
 'bruno-mars.txt',
 'michael-jackson.txt',
 'alicia-keys.txt',
 'missy-elliott.txt',
 'rihanna.txt',
 'leonard-cohen.txt',
 'notorious-big.txt',
 'jimi-hendrix.txt',
 'lil-wayne.txt',
 'dolly-parton.txt']

Voy a elegir las canciones de Nirvana.

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/nirvana.txt', sep='/n', header=None)
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1985


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
sentence_tokens[:5]

[['load', 'up', 'on', 'guns', 'bring', 'your', 'friends'],
 ["it's", 'fun', 'to', 'lose', 'and', 'to', 'pretend'],
 ["she's", 'over', 'bored', 'and', 'self', 'assured'],
 ['oh',
  'no',
  'i',
  'know',
  'a',
  'dirty',
  'word',
  'hello',
  'hello',
  'hello',
  'how',
  'low'],
 ['hello', 'hello', 'hello', 'how', 'low']]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [20]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=50,       # dimensionalidad de los vectores 
                     negative=5,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [23]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [24]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1985


In [25]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 381


### 3 - Entrenar el modelo generador

In [26]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 27366.03515625
Loss after epoch 1: 27527.46875
Loss after epoch 2: 25029.73046875
Loss after epoch 3: 20942.1484375
Loss after epoch 4: 19335.3203125
Loss after epoch 5: 18664.0
Loss after epoch 6: 18523.21875
Loss after epoch 7: 18212.515625
Loss after epoch 8: 17969.15625
Loss after epoch 9: 17914.578125
Loss after epoch 10: 17554.96875
Loss after epoch 11: 17204.046875
Loss after epoch 12: 17085.34375
Loss after epoch 13: 16574.96875
Loss after epoch 14: 16379.96875
Loss after epoch 15: 15874.6875
Loss after epoch 16: 15304.28125
Loss after epoch 17: 14939.1875
Loss after epoch 18: 14471.3125
Loss after epoch 19: 14188.96875
Loss after epoch 20: 13608.84375
Loss after epoch 21: 13210.5
Loss after epoch 22: 12945.875
Loss after epoch 23: 12531.84375
Loss after epoch 24: 12178.3125
Loss after epoch 25: 11895.15625
Loss after epoch 26: 11771.28125
Loss after epoch 27: 11398.40625
Loss after epoch 28: 11001.0625
Loss after epoch 29: 10791.34375
Loss after epoch 30: 1

(1302614, 2469200)

### 4 - Ensayar

In [27]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["hello"], topn=10)

[('low', 0.8179157972335815),
 ('how', 0.7063069939613342),
 ('lights', 0.6345332264900208),
 ('end', 0.5473095178604126),
 ('mind', 0.5197376608848572),
 ('kind', 0.5143505334854126),
 ('old', 0.5040302872657776),
 ('ummmmm', 0.5021024942398071),
 ('hand', 0.49064821004867554),
 ('wounded', 0.488564133644104)]

In [28]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('burlap', 0.343789279460907),
 ('cold', 0.27398481965065),
 ('help', 0.24288813769817352),
 ('died', 0.23086683452129364),
 ('little', 0.228784441947937),
 ('long', 0.22803987562656403),
 ('might', 0.22103741765022278),
 ('driftwood', 0.21550846099853516),
 ('hard', 0.2131664901971817),
 ('fun', 0.21242639422416687)]

In [29]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["albino"], topn=10)

[('mosquito', 0.9298466444015503),
 ('mulatto', 0.917486846446991),
 ('libido', 0.6930466890335083),
 ('complaint', 0.5826694369316101),
 ('new', 0.5516958236694336),
 ('dream', 0.4894977807998657),
 ('luck', 0.46803462505340576),
 ('an', 0.4652242660522461),
 ('wounded', 0.4644226133823395),
 ('fish', 0.4494937062263489)]

In [31]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["mama"], topn=10)

[('surely', 0.6836614012718201),
 ('getting', 0.5527291893959045),
 ('dream', 0.5176275372505188),
 ('yeah', 0.45885926485061646),
 ('leave', 0.4483659565448761),
 ('guess', 0.44214046001434326),
 ('complain', 0.4405381679534912),
 ('sound', 0.39311328530311584),
 ('this', 0.3926202356815338),
 ('off', 0.3814024031162262)]

In [36]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["box"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [37]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [38]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



### 6 - Algunas conclusiones

Con el script estudiado en clase se crean vectores con Gensim empleando el dataset de canciones de Nirvana (nirvana.txt).


Se modifican algunos parámetros del modelo generador de vectores, por ejemplo, tamaño de embeddings de 50, modelo CBOW, frecuencia mínima de 5, etc. Con respecto a la frecuencia mínima, primero se ensayó con 10 pero dejaba afuera a muchas palabras, lo que impactaba negativamente al momento de ver las relaciones.

Se entrenó el modelo con 200 epochs y se analizó la relación de ciertas palabras.

Por ejemplo, la palabra "*hello*" se relaciona positiva y fuertemente con "*how*" y "*low*", como era de esperarse (canción "*Smells like teen spirit*"). De mismo modo, la palabra "*albino*" se relaciona con "*mosquito*", "*mulatto*" y "*libido*".

Otro ensayo fue ver como se relaciona, de manera negativa, la palabra "*love*". Se observó relación negativa con "*cold*", "*died*", "*hard*" lo que es bastante acertado. No queda muy claro porque aparece "*help*", "*fun*" y "*little*".

Se ve que cuando se prueba con alguna palabra que no existe en el vocabulario el script arroja error.

Por último se graficó la agrupación de vectores donde se observan algunos grupos muy fuertemente relacionados (casi un vector arriba del otro), como es el caso de las palabras "*hello*", "*how*" y "*low*", entre otros ejemplos.